In [1]:

%run ../load_magic/dataframes.py
%run ../load_magic/storage.py

s = Storage()
url = 'https://www.globalfirepower.com/countries-listing.php?fbclid=IwAR1s3IrCn2NAYKafbUPkWotw29IDBhZ9e86lyuxF9z3FEKHcA483SiTcTi4'
table_dfs_list = []
html_dir = os.path.join(s.data_folder, 'html')

In [2]:

def get_rank_number(topRow_soup):
    pwrindx_rank = 0
    rankNumContainer_soups_list = topRow_soup.find_all('div', attrs={'class': ['rankNumContainer']})
    for rankNumContainer_soup in rankNumContainer_soups_list:
        pwrindx_rank = int(rankNumContainer_soup.text.strip())
    
    return pwrindx_rank

In [3]:

def get_form_name(countryName_soup, class_name):
    form_name = ''
    formName_soups_list = countryName_soup.find_all('div', attrs={'class': [class_name]})
    for formName_soup in formName_soups_list:
        form_name = formName_soup.text.strip()
    
    return form_name

In [4]:

def get_power_index_score(div_soup):
    pwrindx_score = 0.
    bottomRow_soups_list = div_soup.find_all('div', attrs={'class': ['bottomRow']})
    for bottomRow_soup in bottomRow_soups_list:
        pwrIndxContainer_soups_list = bottomRow_soup.find_all('div', attrs={'class': ['pwrIndxContainer']})
        for pwrIndxContainer_soup in pwrIndxContainer_soups_list:
            pwrindx_score = float(pwrIndxContainer_soup.text.strip().split(': ')[-1])
    
    return pwrindx_score

In [ ]:

try:
    table_dfs_list = get_page_tables(url)
except urllib.error.HTTPError as e:
    print(str(e).strip())
    file_name = 'countries-listing.html'
    file_path = os.path.join(html_dir, file_name)
    try:
        table_dfs_list = get_page_tables(file_path)
    except ValueError as e:
        print(str(e).strip())
        page_soup = get_page_soup(file_path)
        div_soups_list = page_soup.find_all('div', attrs={'class': ['picTrans', 'recordsetContainer', 'boxShadow']})
        rows_list = []
        for div_soup in div_soups_list:
            row_dict = {}
            topRow_soups_list = div_soup.find_all('div', attrs={'class': ['topRow']})
            for topRow_soup in topRow_soups_list:
                row_dict['pwrindx_rank'] = get_rank_number(topRow_soup)
                countryName_soups_list = topRow_soup.find_all('div', attrs={'class': ['countryName']})
                for countryName_soup in countryName_soups_list:
                    row_dict['long_form_name'] = get_form_name(countryName_soup, 'longFormName')
                    row_dict['short_form_name'] = get_form_name(countryName_soup, 'shortFormName')
            row_dict['pwrindx_score'] = get_power_index_score(div_soup)
            
            rows_list.append(row_dict)
except Exception as e:
    print(f'{e.__class__} error: {str(e).strip()}')
firepower_df = pd.DataFrame(rows_list).rename(columns={'long_form_name': 'country_name'})
mask_series = (firepower_df.country_name == 'Vietnam')
firepower_df.loc[mask_series, 'country_name'] = 'Viet Nam'
mask_series = (firepower_df.country_name == 'Syria')
firepower_df.loc[mask_series, 'country_name'] = 'Syrian Arab Republic'
mask_series = (firepower_df.country_name == 'North Korea')
firepower_df.loc[mask_series, 'country_name'] = "Korea (Democratic People's Republic of)"
mask_series = (firepower_df.country_name == 'South Korea')
firepower_df.loc[mask_series, 'country_name'] = 'Korea, Republic of'
mask_series = (firepower_df.country_name == 'Moldova')
firepower_df.loc[mask_series, 'country_name'] = 'Moldova, Republic of'
mask_series = (firepower_df.country_name == 'Iran')
firepower_df.loc[mask_series, 'country_name'] = 'Iran (Islamic Republic of)'
mask_series = (firepower_df.country_name == 'Laos')
firepower_df.loc[mask_series, 'country_name'] = "Lao People's Democratic Republic"
mask_series = (firepower_df.country_name == 'Russia')
firepower_df.loc[mask_series, 'country_name'] = 'Russian Federation'
mask_series = (firepower_df.country_name == 'Taiwan')
firepower_df.loc[mask_series, 'country_name'] = 'Taiwan, Province of China'
mask_series = (firepower_df.country_name == 'United Kingdom')
firepower_df.loc[mask_series, 'country_name'] = 'United Kingdom of Great Britain and Northern Ireland'
s.store_objects(firepower_df=firepower_df)

In [ ]:

%run ../load_magic/lists.py

eurasia_df = s.load_object('eurasia_df')

In [ ]:

typos_df = check_for_typos(eurasia_df.country_name.tolist(), firepower_df.country_name.tolist(), verbose=False)
mask_series = (typos_df.max_similarity < 1.0)
typos_df[mask_series].sort_values(['max_similarity'], ascending=[False])

In [ ]:

typos_df = check_for_typos(firepower_df.country_name.tolist(), eurasia_df.country_name.tolist(), verbose=False)
mask_series = (typos_df.max_similarity < 1.0)
typos_df[mask_series].sort_values(['max_similarity'], ascending=[False])

In [ ]:

eurasia_df = s.load_object('eurasia_df')
eurasia_df = eurasia_df.merge(firepower_df, on=['country_name'], how='left')
s.store_objects(eurasia_df=eurasia_df)

In [3]:

%run ../load_magic/storage.py

s = Storage()
firepower_df = s.load_object('firepower_df')
country_stats_df = s.load_object('country_stats_df')
set(firepower_df.columns).intersection(set(country_stats_df.columns))

{'country_name'}

In [4]:

country_stats_df = s.load_object('country_stats_df')
country_stats_df = country_stats_df.merge(firepower_df, on=['country_name'], how='left')
s.store_objects(country_stats_df=country_stats_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pickle\country_stats_df.pickle
